In [1]:
%%capture
!pip install -q -U "protobuf==3.20.3" "transformers>=4.51.0" datasets accelerate peft trl wandb sacrebleu

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from tqdm import tqdm
from pathlib import Path

BASE_MODEL = "Qwen/Qwen3-1.7B"
CKPT_DIR = Path("/kaggle/input/final-state/checkpoint_last")
MERGED_DIR = "Qwen3-1.7B-medical"

assert os.path.isdir(CKPT_DIR), f"Không thấy checkpoint dir: {CKPT_DIR}"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="sdpa",
)

peft_model = PeftModel.from_pretrained(base, CKPT_DIR)
merged = peft_model.merge_and_unload()

merged.config.use_cache = True
merged.eval()

merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("Saved merged model to:", MERGED_DIR)

2025-12-23 03:31:49.373770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766460709.791930      44 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766460709.896280      44 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766460710.870858      44 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766460710.870887      44 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766460710.870889      44 computation_placer.cc:177] computation placer alr

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Saved merged model to: Qwen3-1.7B-medical


In [ ]:
import unicodedata
from datasets import Dataset

# Hàm làm sạch dòng (giữ nguyên)
def clean_line(s: str) -> str:
    if s is None:
        return ""
    s = s.replace("\ufeff", "")
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    s = s.replace('\u200b', '').replace("\u200e", "")
    s = s.replace("“", '\"').replace("”", '\"')
    s = s.replace("‘", "'").replace("’", "'")
    s = s.replace("‟", '\"')
    s = s.replace("‛", "'")
    
    s = s.replace("bảo hiểm y tế (BHYT)", "bảo hiểm y tế")
    s = s.replace("Bảo hiểm y tế (BHYT)", "bảo hiểm y tế")
    s = s.replace("BHYT", "bảo hiểm y tế")

    s = s.replace("Bệnh nhân (BN)", "bệnh nhân")
    s = s.replace("bệnh nhân (BN)", "bệnh nhân")
    s = s.replace("Bệnh nhân (Bn)", "bệnh nhân")
    s = s.replace("BN", "bệnh nhân")
    
    s = s.replace("nhiễm sắc thể (NST)", "nhiễm sắc thể")
    s = s.replace("NST", "nhiễm sắc thể")
    s = s.replace("khám chữa bệnh (KCB)", "khám chữa bệnh")
    s = s.replace("Khám chữa bệnh (KCB)", "khám chữa bệnh")
    s = s.replace("khám chữa bệnh (kcb)", "khám chữa bệnh")
    s = s.replace("Khám chữa bệnh (kcb)", "khám chữa bệnh")
    s = s.replace("kcb", "khám chữa bệnh")
    s = s.replace("KCB", "khám chữa bệnh")
    
    
    s = " ".join(s.strip().split())
    return s

def read_lines(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return [clean_line(line.rstrip("\n")) for line in f]

# Hàm gộp 10 dòng thành 1 đoạn (giữ nguyên)
def group_lines_into_segments(lines: list[str], N: int = 10) -> list[str]:
    segments = []
    for i in range(0, len(lines), N):
        chunk = lines[i:i + N]
        segments.append("\n".join(chunk))
    return segments

def build_segment_dataset(en_path: str, vi_path: str, N_lines: int = 10):
    en_lines = read_lines(en_path)
    vi_lines = read_lines(vi_path)
    
    if len(en_lines) != len(vi_lines):
        raise ValueError(f"Line count mismatch: en={len(en_lines)} vi={len(vi_lines)}")
        
    total_lines = len(en_lines)
    aligned_total = (total_lines // N_lines) * N_lines
    
    en_segments = group_lines_into_segments(en_lines[:aligned_total], N=N_lines)
    vi_segments = group_lines_into_segments(vi_lines[:aligned_total], N=N_lines)
    
    dropped_truncation = total_lines - aligned_total 

    rows = []
    dropped_empty = 0
    
    for i, (src, tgt) in enumerate(zip(en_segments, vi_segments)):
        if not src.strip() or not tgt.strip():
            dropped_empty += 1
            continue
        rows.append({"en": src, "vi": tgt, "idx": i * N_lines}) 

    ds = Dataset.from_list(rows)
    return ds, dropped_empty * N_lines + dropped_truncation


# TẢI DỮ LIỆU ĐÁNH GIÁ TỪ FILE TEST
eval_ds, dropped_eval = build_segment_dataset(
    "/kaggle/input/test-en2vi/public_test.en.txt",
    "/kaggle/input/test-en2vi/public_test.vi.txt",
    N_lines=20
)

print("Total Eval segments:", len(eval_ds), "Dropped lines (Eval):", dropped_eval)


Total Eval segments: 150 Dropped lines (Eval): 0


In [4]:
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from sacrebleu.metrics import BLEU
from pathlib import Path

MODEL_PATH = Path("/kaggle/working/Qwen3-1.7B-medical") 
MAX_NEW_TOKENS = 1024
N_BLEU_SAMPLES = 150

system_prompt = (
    "You are a medical translation engine. Translate from English to Vietnamese. "
    "Rules: Keep abbreviations as-is (e.g., V.A, V.a, PTA, Type B/C/As). "
    "Preserve all numbers, %, ±, ≥, ≤,... parentheses, and punctuation. "
    "Do not add explanations. Output only the Vietnamese translation."
    "Prioritize medical accuracy and use standard Vietnamese medical terminology."
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B", use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

merged_model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="sdpa",
)
merged_model.eval()
device = merged_model.device
print("Mô hình đã được tải trực tiếp và sẵn sàng để đánh giá.")


# HÀM SINH VĂN BẢN (TRANSLATE)
def translate_one(en: str, model, tokenizer, system_prompt, max_new_tokens=MAX_NEW_TOKENS):
    original_padding_side = tokenizer.padding_side
    tokenizer.padding_side = 'left' 
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Translate English to Vietnamese:\n{en}"},
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=5,
            do_sample=False,
            early_stopping=True,
            temperature=None, 
            top_p=None,
        )
        
    tokenizer.padding_side = original_padding_side
    
    return tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()


# TÍNH TOÁN BLEU SCORE
bleu_metric = BLEU(tokenize="none", effective_order=True)

def compute_bleu(eval_ds, model, tokenizer, system_prompt, n_samples=N_BLEU_SAMPLES):
    
    data = list(eval_ds)
    if n_samples < len(data):
        data = random.sample(data, k=n_samples)

    refs = []
    hyps = []
    i = 1
    for ex in data:
        pred = translate_one(ex["en"], model, tokenizer, system_prompt)
        pred = clean_line(pred)
        
        hyps.append(pred.lower())
        refs.append([ex["vi"].lower()])
        print(i)
        i += 1
        print(pred.lower())
        print(ex["vi"].lower())

    score = bleu_metric.corpus_score(hyps, refs).score
    return score, len(data)

print("Bắt đầu tính BLEU trên tập đánh giá...")
bleu_score, n_used = compute_bleu(eval_ds, merged_model, tokenizer, system_prompt)

print(f"\n - Đánh giá hoàn tất:")
print(f"   - Mô hình: Qwen3-1.7B-LoRA")
print(f"   - BLEU Score (tokenize=none) trên {n_used} đoạn văn: {bleu_score:.2f}")

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Mô hình đã được tải trực tiếp và sẵn sàng để đánh giá.
Bắt đầu tính BLEU trên tập đánh giá...
1
kiến thức, thực hành sử dụng dịch vụ y tế công cộng của người dân và các yếu tố ảnh hưởng tại thành phố vientiane, lào pdr, năm 2017 mô tả thực trạng kiến thức, thực hành sử dụng dịch vụ y tế công cộng của người dân và các yếu tố ảnh hưởng tại thành phố vientiane, lào pdr, năm 2017. phương pháp nghiên cứu: nghiên cứu cắt ngang mô tả được thực hiện trên 928 người dân thuộc diện bảo hiểm y tế (bảo hiểm y tế) người lớn tại 2 huyện phố hồng và keo oudom, thành phố vientiane. kết quả: tỷ lệ người dân có kiến thức đúng về việc sử dụng dịch vụ y tế công cộng (dvytct) miễn phí lần đầu là 44,5% và được cung cấp thông tin bảo hiểm y tế là 34,8%. tỷ lệ người dân có thực hành đi dvytct lần đầu là 61,8%. tỷ lệ người dân có thực hành đi dvytct để nhận thuốc cho người thân / người khác là 20,1%. các yếu tố ảnh hưởng đến kiến thức và thực hành sử dụng dvytct của người dân là khoảng cách và thời gian đi dvyt